# FINAL PROJECT - COLLECT DATA STEP

## Cài đặt môi trường và thư viện

In [4]:
import requests_cache, requests, time, json, re, urllib.parse
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [5]:
requests_cache.install_cache()

## Thập dữ liệu

### BILLBOARD HOT 100
**Docs:** https://foundation.wikimedia.org/wiki/Terms_of_Use/en
- Thu thập thông tin tên và thứ hạng của bài hát từ Wikipedia (từ 2005 đến 2021)

In [6]:
years = list(range(2005,2021))

ranks = pd.DataFrame(None)
for year in years:
    url = f'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{year}'

    response = requests.get(url)
    if response.from_cache:
        print('Get data from cache')
    else: 
        print('Sleep 1s')
        time.sleep(1)
    
    tree = BeautifulSoup(response.text)
    table = tree.find('table', {'class': 'wikitable sortable'})
    songs = table.find_all('tr')
    infors = []
    for song in songs:
        infors.append(song.text.replace('"', '').split('\n')[1:-1])
    rank = pd.DataFrame(infors[1:], columns = ['rank', 'name', 'artist'])
    rank['rank'] = rank['rank'].astype('int')
    rank['year'] = year
    ranks = pd.concat([ranks, rank], ignore_index = True)

Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache


In [7]:
ranks

,rank,name,artist,year
0,1,We Belong Together,Mariah Carey,2005
1,2,Hollaback Girl,Gwen Stefani,2005
2,3,Let Me Love You,Mario,2005
3,4,Since U Been Gone,Kelly Clarkson,2005
4,5,"1, 2 Step",Ciara featuring Missy Elliott,2005
...,...,...,...,...
1595,96,More Than My Hometown,Morgan Wallen,2020
1596,97,Lovin' on You,Luke Combs,2020
1597,98,Said Sum,Moneybagg Yo,2020
1598,99,Slide,H.E.R. featuring YG,2020


### Spotify API
**Docs :** https://developer.spotify.com/documentation/web-api/

- Đầu tiên, Chúng ta sử dụng API Search để thu thập các Items dựa trên tên và nghệ sĩ của bản nhạc ở trên. Chúng ta nhận được ID track, tracks name, artists ID, name artists, available market,...

- Sau đó, Chúng ta sử dụng API Track để lấy Audio Feature của Track dựa trên ID track được tìm kiếm ở trên. Chúng ta nhận được một số thuộc tính của bản nhạc như acousticness, danceability, energy,...

- Tiếp theo, Chúng ta sử dụng API Artist để tìm kiếm thông tin về Nghệ sĩ dựa trên artists ID ở trên. Chúng ta nhận được name artist, population artist, artist followers,...

- Cuối cùng, Chúng ta tổng hợp thông tin các bản nhạc và tạo DataFrame cho phần sau

In [8]:
client_id = 'YOUR-CLIENT-ID'
client_secret = 'YOUR-CLIENT-SECRET'

token_url = 'https://accounts.spotify.com/api/token'
token_auth = {'grant_type': 'client_credentials',
              'client_id': client_id,
              'client_secret': client_secret,}
r = requests.post(token_url, token_auth)
data = r.json()
access_token = data['access_token']

df = ranks[['name', 'artist']]


In [9]:
headers_params = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}
base_url = f'https://api.spotify.com/v1/'


def replace_name(name):
  name = name.replace(" & ", ' and ')
  name = name.replace('...', ' ')
  name = name.replace(".", '')
  name = name.replace(",", '')
  name = name.replace("'", '')
  name = name.replace('(', '')
  name = name.replace(')', '')
  name = name.replace(" - ", ' ')
  name = name.replace(' – ', ' ')
  name = name.replace('/', ' ')
  name = name.replace('   ', ' ')
  name = name.replace('  ', ' ')
  return name

def track_info(col):
    

    query_str = col['name'].replace('.', '').replace(" & ", ' and ').replace(",", '').replace("'", '') + ' ' + re.sub('featuring.*|and.*|,.*', '', col['artist'])
    query_str = query_str.replace("Big Shit Poppin", "Big S**t Poppin")
    query_str = query_str.replace("Juju on That Beat (TZ Anthem)", "Juju on That Beat")
    query_str = query_str.replace("Savage Love (Laxed – Siren Beat)", "Savage Love")
    query_str = query_str.replace("10000 Hours Dan + Shay ", "10,000 Hours")
    query_str = query_str.replace("Bust It Baby (Part 2)", "Bust It Baby")
    query_str = query_str.replace("Niggas in Paris", "Ni**as in Paris")
    
    params = {'q': query_str, 'type': 'track'}
    artist_name = np.nan
    pop_artist = np.nan
    artist_followers = np.nan
    track_name = np.nan
    available_market = np.nan
    pop_track = np.nan
    acousticness = np.nan
    danceability = np.nan
    duration_ms = np.nan
    energy = np.nan
    instrumentalness = np.nan
    liveness = np.nan
    loudness = np.nan
    speechiness = np.nan
    tempo = np.nan
    time_signature = np.nan
    valence = np.nan

    try:
        #Request bai hat va ten ca si
        json_str = requests.get(base_url + 'search/?limit=50', headers = headers_params, params = params)
        infors = json.loads(json_str.text)
        infor = infors['tracks']['items']
        #Dua vao thong tin request de lay du lieu tong quat
        if infor == []:
            print('Cant find this song in spotify')
            print(col['name'])
            print(query_str)
            return artist_name, pop_artist, artist_followers, track_name, available_market, pop_track, acousticness, danceability, \
                duration_ms, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, valence

        col_name = str
        index = len(infor)+1
        flag = 0
        for i in range(0,len(infor)):
          col_name = col['name'].lower()
          #Truong hop can sua lai
          col_name = replace_name(col_name)
          #Truong hop dac biet(kiem tra bang mat)
          col_name = col_name.replace("1 2", 'one two')
          col_name = col_name.replace("obsession no es amor", 'obsesion no es amor')
          col_name = col_name.replace("unpredictable", 'unpredictable main')
          col_name = col_name.replace("i think they like me", 'i think they like me remix')
          col_name = col_name.replace("cupids chokehold", 'cupids chokehold breakfast in america')
          col_name = col_name.replace("big shit poppin do it", "big s**t poppin [do it]")
          col_name = col_name.replace("thats that", 'thats that shit')
          col_name = col_name.replace("rockin that thang", 'rockin that thang album version edited')
          if col_name == "how do you sleep?" and re.sub('featuring.*|and.*|,.*', '', col['artist']) == "Jesse McCartney ":
            col_name = col_name.replace("how do you sleep?", 'how do you sleep? originally performed by jesse mccartney')
          col_name = col_name.replace("niggas in paris", 'ni**as in paris')
          col_name = col_name.replace("gangnam style", 'gangnam style 강남스타일')
          col_name = col_name.replace("back in time", 'back in time featured in "men in black 3"')
          col_name = col_name.replace("cant hold us", 'cant hold us feat ray dalton')
          col_name = col_name.replace("cups pitch perfects when im gone", 'cups pitch perfect’s “when i’m gone” pop version')
          col_name = col_name.replace("fuckin problems", 'f**kin problems')
          col_name = col_name.replace("same love", 'same love feat mary lambert')
          col_name = col_name.replace("bailando", 'bailando english version')
          col_name = col_name.replace("watch me", 'watch me whip nae nae')
          col_name = col_name.replace("where are ü now", 'where are ü now with justin bieber')
          col_name = col_name.replace("hit the quan", 'hit the quan original version')
          col_name = col_name.replace("el perdón", 'el perdón with enrique iglesias')
          col_name = col_name.replace("sucker for pain", 'sucker for pain with wiz khalifa imagine dragons logic and ty dolla $ign feat x ambassadors')
          col_name = col_name.replace("juju on that beat tz anthem", 'juju on that beat')
          col_name = col_name.replace("ric flair drip", 'ric flair drip & metro boomin')
          if ((col_name == "baby") and (re.sub('featuring.*|and.*|,.*', '', col['artist']) == "Lil Baby ")):
            col_name = "baby lil baby feat dababy"
          if col_name == "ballin":
            col_name = "ballin with roddy ricch"
          col_name = col_name.replace("nobody but you", 'nobody but you duet with gwen stefani')
          col_name = col_name.replace("pussy fairy otw", 'p*$$y fairy otw')
          col_name = col_name.replace("death bed", 'death bed coffee for your head')



          col_name = col_name.replace("too little too late", "too little too late 2018")
          col_name = col_name.replace("déjà vu", "deja vu")
          col_name = col_name.replace("im a flirt", "im a flirt remix")
          col_name = col_name.replace("lovestoned", "lovestoned i think she knows interlude")
          col_name = col_name.replace("dont you wanna stay", "dont you wanna stay with kelly clarkson")
          col_name = col_name.replace("somethin bad", "somethin bad with carrie underwood duet with carrie underwood")
          col_name = col_name.replace("earned it", "earned it fifty shades of grey")
          col_name = col_name.replace('cant stop the feeling!','cant stop the feeling! original song from dreamworks animations "trolls"')
          if col_name == 'more':
            col_name = 'more redone jimmy joker remix'
          if col_name == "let it go" and re.sub('featuring.*|and.*|,.*', '', col['artist']) == "Idina Menzel":
            col_name = 'let it go from "frozen"'
          col_name = col_name.replace("bust it baby part 2", "bust it baby pt 2")
          #col_name = col_name.replace("these words", "these words")
          col_name = col_name.replace("remind me", "remind me with carrie underwood")
          col_name = col_name.replace("just like fire", 'just like fire from the original motion picture "alice through the looking glass"')
          col_name = col_name.replace("i dont wanna live forever", 'i don’t wanna live forever fifty shades darker')
          col_name = col_name.replace("all the stars", 'all the stars with sza')
          col_name = col_name.replace("powerglide", 'powerglide from sr3mm')
          col_name = col_name.replace("sunflower", 'sunflower spider-man: into the spider-verse')
          col_name = col_name.replace("i dont care", 'i dont care with justin bieber')
          col_name = col_name.replace("10000 hours", '10000 hours with justin bieber')
          col_name = col_name.replace("be like that", 'be like that feat swae lee and khalid')
          
          
          name = re.sub(r' \(feat.*?\)', '', infor[i]['name'].lower())
          name = replace_name(name)
          name = name.replace(' - radio edit', '')
          name = name.replace(' - pop version', '')
          name = name.replace(' - original clean', '')
          name = name.replace(' - encore', '')
          name = name.replace(' personal', '')

          if col_name == name:
            track_id = infor[i]['id']
            audio_features = requests.get(base_url + 'audio-features/' + track_id, headers=headers_params)
            audio_info = audio_features.json()
            if 'error' not in audio_info.keys():
              index = i
              break
            else: 
              print('Cant find audio')
              print(infor[i]['name'] + ' ' + col['artist'])
              print(base_url + 'audio-features/' + track_id)
              print(audio_info['error']['message'])
              print(infor[i]['id'])

        if index == (len(infor)+1):
          print('Cant find song. Not the same name')
          print(col['name'] + ' ' + re.sub(' featuring.*|and.*|,.*', '', col['artist']))
          print(col_name)
          print(re.sub(' featuring.*|and.*|,.*', '', col['artist']))
          return artist_name, pop_artist, artist_followers, track_name, available_market, pop_track, acousticness, danceability, \
                duration_ms, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, valence


        track_name = infor[index]['name']
        available_market = len(infor[index]['available_markets'])
        pop_track = infor[index]['popularity']

        #Dua vao request de lay du lieu bai hat
        acousticness = audio_info['acousticness']
        danceability = audio_info['danceability']
        duration_ms = audio_info['duration_ms']
        energy = audio_info['energy']
        instrumentalness = audio_info['instrumentalness']
        liveness = audio_info['liveness']
        loudness = audio_info['loudness']
        speechiness = audio_info['speechiness']
        tempo = audio_info['tempo']
        time_signature = audio_info['time_signature']
        valence = audio_info['valence']

        #Request chinh xac ca si
        artist_id = infor[index]['artists'][0]['id']
        artist_infor = requests.get(base_url + 'artists/' + artist_id, headers = headers_params)
        artist_info = artist_infor.json()

        #Dua vao request de lay du lieu ca si
        artist_name = artist_info['name']
        pop_artist = artist_info['popularity']
        artist_followers = artist_info['followers']['total']
        
        if json_str.from_cache:
            print('Get data from cache')
            pass
        else: 
            print('Sleep 1s')
            time.sleep(1)

        return artist_name, pop_artist, artist_followers, track_name, available_market, pop_track, acousticness, danceability, \
                duration_ms, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, valence

    except:
        print("Error Runtime")
        return artist_name, pop_artist, artist_followers, track_name, available_market, pop_track, acousticness, danceability, \
                duration_ms, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, valence

In [10]:
df = ranks[['name', 'artist']]
df = df[df['name']== "Savage Love (Laxed – Siren Beat)"]
df['artist_name'], df['pop_artist'], df['artist_followers'], df['track_name'], df['available_market'], df['pop_track'], \
    df['acousticness'], df['danceability'], df['duration_ms'], df['energy'], df['instrumentalness'], \
    df['liveness'], df['loudness'], df['speechiness'], df['tempo'], df['time_signature'], df['valence'] \
    = zip(*df.apply(track_info, axis = 1))

Get data from cache


In [11]:
df = ranks[['name', 'artist']]
df['artist_name'], df['pop_artist'], df['artist_followers'], df['track_name'], df['available_market'], df['pop_track'], \
    df['acousticness'], df['danceability'], df['duration_ms'], df['energy'], df['instrumentalness'], \
    df['liveness'], df['loudness'], df['speechiness'], df['tempo'], df['time_signature'], df['valence'] \
    = zip(*df.apply(track_info, axis = 1))

Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Cant find this song in spotify
Hate It or Love It
Hate It or Love It The Game 
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
G

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df.to_csv('audio_artist.csv')
df

,name,artist,artist_name,pop_artist,artist_followers,track_name,available_market,pop_track,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence
0,We Belong Together,Mariah Carey,Mariah Carey,78.0,6995024.0,We Belong Together,3.0,69.0,0.03390,0.837,201400.0,0.462,0.000000,0.0900,-7.890,0.0601,139.986,4.0,0.762
1,Hollaback Girl,Gwen Stefani,Gwen Stefani,75.0,4859586.0,Hollaback Girl,165.0,68.0,0.35000,0.926,199853.0,0.916,0.000006,0.0234,-2.221,0.0929,110.007,4.0,0.904
2,Let Me Love You,Mario,Mario,67.0,2367100.0,Let Me Love You,143.0,72.0,0.23500,0.656,256733.0,0.578,0.000000,0.1180,-8.970,0.0922,94.514,4.0,0.556
3,Since U Been Gone,Kelly Clarkson,Kelly Clarkson,75.0,7955803.0,Since U Been Gone,178.0,70.0,0.00165,0.662,188960.0,0.741,0.030300,0.1140,-5.406,0.0334,131.000,4.0,0.404
4,"1, 2 Step",Ciara featuring Missy Elliott,Ciara,72.0,5209750.0,"One, Two Step (feat. Missy Elliott)",178.0,73.0,0.04200,0.944,203787.0,0.511,0.001070,0.0379,-10.960,0.1610,113.046,4.0,0.842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,More Than My Hometown,Morgan Wallen,Morgan Wallen,83.0,2238350.0,More Than My Hometown,178.0,74.0,0.61500,0.620,216573.0,0.869,0.000000,0.1310,-5.479,0.0462,126.019,4.0,0.597
1596,Lovin' on You,Luke Combs,Luke Combs,82.0,5474485.0,Lovin' On You,178.0,69.0,0.00165,0.572,194867.0,0.949,0.000195,0.1630,-4.865,0.0600,118.974,4.0,0.530
1597,Said Sum,Moneybagg Yo,Moneybagg Yo,80.0,2307788.0,Said Sum,178.0,69.0,0.01850,0.929,155168.0,0.667,0.000000,0.1000,-6.789,0.3530,126.998,4.0,0.274
1598,Slide,H.E.R. featuring YG,H.E.R.,79.0,4240318.0,Slide,177.0,67.0,0.08070,0.832,238321.0,0.469,0.000008,0.2070,-9.141,0.3390,97.023,4.0,0.197


### Genius API
**Docs:** https://docs.genius.com/

- Sử dụng API Search để tìm kiếm bài hát từ tên bài hát + tên ca sĩ trên nền tảng Genius. Từ đây, lấy được link của lời bài hát.
- Sau đó, request đến đường link vừa nhận được và thực hiện lấy lời bài hát về.

In [13]:
genius = ranks[['name', 'artist']]
genius

,name,artist
0,We Belong Together,Mariah Carey
1,Hollaback Girl,Gwen Stefani
2,Let Me Love You,Mario
3,Since U Been Gone,Kelly Clarkson
4,"1, 2 Step",Ciara featuring Missy Elliott
...,...,...
1595,More Than My Hometown,Morgan Wallen
1596,Lovin' on You,Luke Combs
1597,Said Sum,Moneybagg Yo
1598,Slide,H.E.R. featuring YG


In [14]:
def compare_name(a, b):
    return [c for c in a if c.isalpha()] == [c for c in b if c.isalpha()]

def get_lyrics(col):
    query_str = col['name'] + ' ' + re.sub('featuring.*|and.*|,.*', '', col['artist'])

    #Specialize for some cases
    query_str = query_str.lower().replace('the black eyed peas', 'black eyed peas')
    query_str = query_str.replace('f**kin', 'fuckin')
    query_str = query_str.replace('watch me silentó', 'watch me (whip / nae nae) silentó')
    query_str = query_str.replace('baby lil baby', 'baby quality control lil baby')
    query_str = re.sub(r'\(pitch .*?\)', '', query_str)
    query_str = query_str.strip(" ")

    url = 'https://api.genius.com/search?q=' + urllib.parse.quote(query_str)
    ACCESS_TOKEN_GENIUS = 'YOUR-GENIUS-ACCESS-TOKEN'

    params = {
        'User-Agent': 'CompuServe Classic/1.22',
        'Authorization': f'Bearer {ACCESS_TOKEN_GENIUS}'
    }    
    json_str = requests.get(url, headers = params)
    infors = json.loads(json_str.text)

    lyrics_url = ''
    lyrics_title = ''
    for song in infors['response']['hits']:
        #Modify lyrics tile: and -> &
        modify_title = song['result']['title'].replace('and', '&')
        #Modify song's name: and -> &
        modify_sname = col['name'].replace('and', '&')

        if compare_name(modify_sname.lower(), modify_title.lower()):          
            lyrics_url = song['result']['url']
            lyrics_title = song['result']['title']
            break
    
    #If we can't find the exact same song title, try some modifications
    if lyrics_url == '':
        for song in infors['response']['hits']:
            ignore_list = ['español', 'remix', '8-bit']
            if not any(x in song['result']['title'].lower() for x in ignore_list):
                #Modify lyrics title: delete inside ()
                modify_title = re.sub(r' \(.*?\)', '', song['result']['title'].lower())
                #Modify song's name: delete inside ()
                modify_sname = re.sub(r'\(Pitch .*?\)', '', col['name'])
                
                if compare_name(modify_sname.lower(), modify_title.lower()):          
                    lyrics_url = song['result']['url']
                    lyrics_title = song['result']['title']
                    break

                #Modify lyrics title: remove after /
                modify_title = re.sub('/.*', '', song['result']['title'].lower())
                if compare_name(col['name'].lower(), modify_title.lower()):          
                    lyrics_url = song['result']['url']
                    lyrics_title = song['result']['title']
                    break

                #Modify lyrics title: remove outside ()
                modify_title = re.findall(r' \(.*?\)', song['result']['title'].lower())
                for m_title in modify_title:
                    if compare_name(col['name'].lower(), m_title.lower()):
                        lyrics_url = song['result']['url']
                        lyrics_title = song['result']['title']
                        break

                #Specialize for some title:
                modify_sname = col['name'].lower().replace('f**kin', 'fuckin')
                modify_title = song['result']['title'].lower()
                modify_title = modify_title.replace('shit', '')
                modify_title = modify_title.replace('pu$$y', 'pussy')
                modify_title = modify_title.replace('mía', 'mia')

                if compare_name(modify_sname, modify_title):          
                    lyrics_url = song['result']['url']
                    lyrics_title = song['result']['title']
                    break
                
            
    if lyrics_url != '':
        response = requests.get(lyrics_url)
        if response.from_cache:
            print('Get data from cache')
        else: 
            print('Sleep 1s')
            time.sleep(1)

        html_text = response.text.replace('<br/>', '\n')
        tree_ = BeautifulSoup(html_text, 'html.parser')
        
        try:
            lyrics_text = tree_.find("div", class_ = re.compile("^lyrics$|Lyrics__Root")).get_text()
        except AttributeError:
            print(lyrics_title)
            print(lyrics_url)

        if lyrics_text is not None:
            #Remove [Intro], [Verse 1]...
            lyrics_text = re.sub(r'\[.*?\]', '', lyrics_text)

            #Remove () brackets
            lyrics_text = lyrics_text.replace('(', '')
            lyrics_text = lyrics_text.replace(')', '')

            #Remove the gaps between 2 parts, and replace with a space
            lyrics_text = re.sub(r'\n{2}', '\n', lyrics_text)
            lyrics_text = re.sub(r'\n', ' ', lyrics_text)
            lyrics_text = lyrics_text.rstrip('EmbedShare URLCopyEmbedCopy')
            lyrics_text = lyrics_text.rstrip('1234567890')
            return lyrics_title, lyrics_text.strip(" ")
    
    print("Can't find lyrics of this song: ", col['name'], " by ", col['artist'])
    cannot.append(query_str)
    return None

In [15]:
genius['title'], genius['lyrics'] = zip(*genius.apply(get_lyrics, axis = 1))

Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache
Get data from cache


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
genius.to_csv('lyrics.csv')
genius

,name,artist,title,lyrics
0,We Belong Together,Mariah Carey,We Belong Together,"Sweet love, yeah I didn't mean it when I said..."
1,Hollaback Girl,Gwen Stefani,Hollaback Girl,"Uh-huh, this my shit All the girls, stomp your..."
2,Let Me Love You,Mario,Let Me Love You,"Mmmm, ah Mmmm, yeah Mmmm Yeah, yeah, yeah Mmmm..."
3,Since U Been Gone,Kelly Clarkson,Since U Been Gone,"Here's the thing, we started out friends It wa..."
4,"1, 2 Step",Ciara featuring Missy Elliott,"1, 2 Step",Ladies and Gentlemen Ladies and Gentlemen This...
...,...,...,...,...
1595,More Than My Hometown,Morgan Wallen,More Than My Hometown,"Girl, our mamas are best friends and so are we..."
1596,Lovin' on You,Luke Combs,Lovin’ on You,Don't get me wrong I like a bobber on the wate...
1597,Said Sum,Moneybagg Yo,Said Sum,"Turn me up, YC Huh? What? Ah, I thought a bro..."
1598,Slide,H.E.R. featuring YG,Slide,You always wearin' them glasses You don't wann...


### Tổng hợp dữ liệu để tạo Dataset

In [18]:
data = pd.concat([ranks, df.drop(['name', 'artist', 'artist_name', 'track_name'], axis = 1), genius.drop(['name', 'artist', 'title'], axis = 1)], axis = 1)
data.to_csv('data.csv', index = False)
data

,rank,name,artist,year,pop_artist,artist_followers,available_market,pop_track,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,lyrics
0,1,We Belong Together,Mariah Carey,2005,78.0,6995024.0,3.0,69.0,0.03390,0.837,201400.0,0.462,0.000000,0.0900,-7.890,0.0601,139.986,4.0,0.762,"Sweet love, yeah I didn't mean it when I said..."
1,2,Hollaback Girl,Gwen Stefani,2005,75.0,4859586.0,165.0,68.0,0.35000,0.926,199853.0,0.916,0.000006,0.0234,-2.221,0.0929,110.007,4.0,0.904,"Uh-huh, this my shit All the girls, stomp your..."
2,3,Let Me Love You,Mario,2005,67.0,2367100.0,143.0,72.0,0.23500,0.656,256733.0,0.578,0.000000,0.1180,-8.970,0.0922,94.514,4.0,0.556,"Mmmm, ah Mmmm, yeah Mmmm Yeah, yeah, yeah Mmmm..."
3,4,Since U Been Gone,Kelly Clarkson,2005,75.0,7955803.0,178.0,70.0,0.00165,0.662,188960.0,0.741,0.030300,0.1140,-5.406,0.0334,131.000,4.0,0.404,"Here's the thing, we started out friends It wa..."
4,5,"1, 2 Step",Ciara featuring Missy Elliott,2005,72.0,5209750.0,178.0,73.0,0.04200,0.944,203787.0,0.511,0.001070,0.0379,-10.960,0.1610,113.046,4.0,0.842,Ladies and Gentlemen Ladies and Gentlemen This...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,96,More Than My Hometown,Morgan Wallen,2020,83.0,2238350.0,178.0,74.0,0.61500,0.620,216573.0,0.869,0.000000,0.1310,-5.479,0.0462,126.019,4.0,0.597,"Girl, our mamas are best friends and so are we..."
1596,97,Lovin' on You,Luke Combs,2020,82.0,5474485.0,178.0,69.0,0.00165,0.572,194867.0,0.949,0.000195,0.1630,-4.865,0.0600,118.974,4.0,0.530,Don't get me wrong I like a bobber on the wate...
1597,98,Said Sum,Moneybagg Yo,2020,80.0,2307788.0,178.0,69.0,0.01850,0.929,155168.0,0.667,0.000000,0.1000,-6.789,0.3530,126.998,4.0,0.274,"Turn me up, YC Huh? What? Ah, I thought a bro..."
1598,99,Slide,H.E.R. featuring YG,2020,79.0,4240318.0,177.0,67.0,0.08070,0.832,238321.0,0.469,0.000008,0.2070,-9.141,0.3390,97.023,4.0,0.197,You always wearin' them glasses You don't wann...
